<a href="https://colab.research.google.com/github/danthonyrodriguez/Colab-Notebooks/blob/main/Copy_of_cyber_intesity_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
%matplotlib inline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [2]:
from google.colab import files
data_to_load = files.upload()

Saving Logistic Regression Testing.csv to Logistic Regression Testing (2).csv


In [2]:
#Created new csv from "merge_wc_edit_update.csv". Removed Q31-Q34 as those are now the "max" variables and targets. Also removed several columns that seemed to be added from the original analysis
data = pd.read_csv('Logistic Regression Testing (2).csv',na_values=[9999, '', ' ']) #read data , label 9999 and blank as missing

In [70]:
data.head()

,Q1,Q2,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,Q3,Q4_1,Q4_2,Q4_3,Q4_4,Q4_5,Q4_6,Q4_7,Q4_8,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17_1,Q17_2,Q17_3,Q17_4,Q17_5,Q17_6,Q18,Q19,Q20,Q21_1,...,Q42,Age,Region,Q45,PhysicalActionsHomeland,PhysicalActionsBagaria,CyberActionsHomeland,CyberActionsBagaria,US_dummy,Taiwan_dummy,SouthAfrica_dummy,filter_$,hacking_scale,gender_rec,binary_guess_pass,binary_access_account_view_files,binary_add_delete_change,binary_malware_use_delete,binary_private_harassment,binary_malware_use_access,binary_public_harassment,binary_write_malware,CyberTrespass,GAvai_FB,Total_Hour,Male,phy_foreign_target,cbr_foreign_target,phy_home_target,cbr_home_target,ENI_Factors,BGE_Factors,OGA_Factors,TECH_Factors,PIR_Factors,cbr_home_intensity,cbr_foreign_intensity,phy_home_intensity,phy_foreign_intensity,cyber_high_target
0,3,12,1,1,0,0,0,0,0.0,3,4,4,4,3,2,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1.0,NaN,1.0,1.0,0.0,1.0,1.0,0,1,0,1,3.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,5.0,NaN,1,2,2,2,0.410557,1.365297,0.066123,1.690926,1.320077,medium,medium,medium,low,0
1,2,1,1,0,0,0,0,0,0.0,3,5,5,5,5,4,4,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,0,1,0,1,3.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,7.0,NaN,2,2,2,2,-1.612441,8.128382,2.822864,-2.622617,-0.511065,medium,medium,medium,medium,0
2,2,13,1,0,1,0,0,0,0.0,2,3,4,2,2,2,2,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1.0,0.0,1.0,1.0,3.0,1.0,1.0,0,1,0,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,6.0,0.0,5,2,2,2,0.815450,0.673172,-1.527111,2.153578,0.296094,medium,medium,medium,high,0
3,3,3,0,0,1,0,0,0,0.0,3,4,4,4,2,3,3,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1.0,NaN,1.0,2.0,2.0,1.0,1.0,0,1,0,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,5.0,0.0,3,2,3,2,0.469872,-1.689587,-0.095561,-0.398677,-1.908558,medium,medium,medium,medium,0
4,2,1,1,0,0,0,0,0,0.0,2,5,4,1,2,3,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,1.0,NaN,1.0,2.0,1.0,0.0,0.0,0,1,0,1,12.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,17.0,NaN,6.0,0.0,2,0,8,0,2.409520,2.858529,-1.960660,2.564042,-1.101400,low,low,high,medium,0


In [3]:
#Add 4 max variables created in exercise 1 of this project
data = data.rename({'Q31_Max': 'phy_foreign_target','Q33_Max': 'phy_home_target','Q32_Max': 'cbr_foreign_target','Q34_Max': 'cbr_home_target'}, axis=1)
data.head()

,Q1,Q2,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,Q3,Q4_1,Q4_2,Q4_3,Q4_4,Q4_5,Q4_6,Q4_7,Q4_8,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17_1,Q17_2,Q17_3,Q17_4,Q17_5,Q17_6,Q18,Q19,Q20,Q21_1,...,Q37_10,Gender,Q39,Q40,Q41,Q42,Age,Region,Q45,PhysicalActionsHomeland,PhysicalActionsBagaria,CyberActionsHomeland,CyberActionsBagaria,US_dummy,Taiwan_dummy,SouthAfrica_dummy,filter_$,hacking_scale,gender_rec,binary_guess_pass,binary_access_account_view_files,binary_add_delete_change,binary_malware_use_delete,binary_private_harassment,binary_malware_use_access,binary_public_harassment,binary_write_malware,CyberTrespass,GAvai_FB,Total_Hour,Male,phy_foreign_target,cbr_foreign_target,phy_home_target,cbr_home_target,ENI_Factors,BGE_Factors,OGA_Factors,TECH_Factors,PIR_Factors
0,3,12,1,1,0,0,0,0,0.0,3,4,4,4,3,2,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,1,NaN,5,1,1.0,NaN,1.0,1.0,0.0,1.0,1.0,0,1,0,1,3.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,5.0,NaN,1,2,2,2,0.410557,1.365297,0.066123,1.690926,1.320077
1,2,1,1,0,0,0,0,0,0.0,3,5,5,5,5,4,4,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,0,NaN,0,0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,0,1,0,1,3.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,7.0,NaN,2,2,2,2,-1.612441,8.128382,2.822864,-2.622617,-0.511065
2,2,13,1,0,1,0,0,0,0.0,2,3,4,2,2,2,2,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,1.0,3,NaN,1,1,1.0,0.0,1.0,1.0,3.0,1.0,1.0,0,1,0,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,6.0,0.0,5,2,2,2,0.815450,0.673172,-1.527111,2.153578,0.296094
3,3,3,0,0,1,0,0,0,0.0,3,4,4,4,2,3,3,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,1.0,3,NaN,1,1,1.0,NaN,1.0,2.0,2.0,1.0,1.0,0,1,0,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,NaN,5.0,0.0,3,2,3,2,0.469872,-1.689587,-0.095561,-0.398677,-1.908558
4,2,1,1,0,0,0,0,0,0.0,2,5,4,1,2,3,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,1.0,5,NaN,2,2,1.0,NaN,1.0,2.0,1.0,0.0,0.0,0,1,0,1,12.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,17.0,NaN,6.0,0.0,2,0,8,0,2.409520,2.858529,-1.960660,2.564042,-1.101400


In [ ]:
#imp_vars =['Q39', 'Q40', 'Gender', 'Q41', 'Q42', 'Q44', 'Age', 'Region','Q14','Q5', 'Q2','Q23', 'Q24', 'Q25', 'Q27', 'Q28', 
#           'GroupEquality', 'OutgroupAntagonism', 'TechnologicalSkill', 'Piracy', 'Nationalism', 'Patriotism', 'cbr_home_target', 'cbr_foreign_target']

In [4]:
#Renaming data frame to utilize Hiwot's bas code for the logistic regression model
dumy2=data

In [5]:
dumy2 = dumy2.round(2)

In [6]:
dumy2 = dumy2.fillna(0)

In [7]:
# This function is used to create a cyber home attack intensity target from cyber home attack as low, medium and high, three classes
def createCyberIntensity(row, col_name):
  if(row[col_name]<=1):
    return 'low'
  if(row[col_name] >1 and row[col_name]<=3):
    return 'medium'
  else:
    return 'high'

Low = cyber attack of 1 or 0, 0 is for missing and 1 is for do nothing

medium = cyber attack of 2 or 3, 2 is for post a facebook or deface a personal website

High = cyber attack >=4 

In [8]:
dumy2['cbr_home_intensity'] =dumy2.apply (lambda row: createCyberIntensity(row, 'cbr_home_target'), axis=1)
dumy2['cbr_foreign_intensity'] =dumy2.apply (lambda row: createCyberIntensity(row, 'cbr_foreign_target'), axis=1)
dumy2['phy_home_intensity'] =dumy2.apply (lambda row: createCyberIntensity(row, 'phy_home_target'), axis=1)
dumy2['phy_foreign_intensity'] =dumy2.apply (lambda row: createCyberIntensity(row, 'phy_foreign_target'), axis=1)

In [9]:
#this function is used to create a binary (0 or 1 ) target variable . for example , high cyber attack =1 else 0. 
def createTarget(row, col_name):
  if(row[col_name]=='high'):
    return 1
  else:
    return 0

In [35]:
#creating a binary variable . 
dumy2['cyber_home_target'] =dumy2.apply (lambda row: createTarget(row, 'cbr_home_intensity'), axis=1)
dumy2['cyber_foreign_target'] =dumy2.apply (lambda row: createTarget(row, 'cbr_foreign_intensity'), axis=1)
dumy2['phy_home_target'] =dumy2.apply (lambda row: createTarget(row, 'phy_home_intensity'), axis=1)
dumy2['phy_foreign_target'] =dumy2.apply (lambda row: createTarget(row, 'phy_foreign_intensity'), axis=1)

In [11]:
dumy2 = dumy2.rename({'Q39': 'educ_level','Q40': 'name_univ','Q41': 'emp_status','Q42': 'marital_status','Q44': 'country','Q14': 'fb_profile','Q5': 'share_comp','Q2': 'num_os'}, axis=1)  # renaming varialbles

In [12]:
#Initial variable selection
pred_vars_all =['educ_level', 'Gender', 'emp_status', 'marital_status', 'Age', 'Region', 
       'ENI_Factors', 'BGE_Factors','OGA_Factors', 'TECH_Factors', 'PIR_Factors']

In [ ]:
#pred_vars1=['Gender', 'Age','Region','GroupEquality', 'OutgroupAntagonism','TechnologicalSkill', 'Piracy', 'Nationalism', 'Patriotism']

Logistic Regression for Cyber Home Attack

In [13]:
X = dumy2[pred_vars_all]
X.shape

(1099, 11)

In [14]:
y = dumy2['cyber_home_target']
y.shape

(1099,)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
logmodel = LogisticRegression(max_iter=1000)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [17]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logmodel.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.76


In [18]:
confusion_matrix(y_test, predictions)

array([[166,   2],
       [ 51,   1]])

In [19]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.76      0.99      0.86       168
           1       0.33      0.02      0.04        52

    accuracy                           0.76       220
   macro avg       0.55      0.50      0.45       220
weighted avg       0.66      0.76      0.67       220



Logistic Regression for Cyber Foreign Attack

In [34]:
dumy2['cbr_foreign_intensity'].value_counts()

medium    512
low       299
high      288
Name: cbr_foreign_intensity, dtype: int64

In [32]:
dumy2.head()

,Q1,num_os,Q2_1,Q2_2,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,Q3,Q4_1,Q4_2,Q4_3,Q4_4,Q4_5,Q4_6,Q4_7,Q4_8,share_comp,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,fb_profile,Q15,Q16,Q17_1,Q17_2,Q17_3,Q17_4,Q17_5,Q17_6,Q18,Q19,Q20,Q21_1,...,Age,Region,Q45,PhysicalActionsHomeland,PhysicalActionsBagaria,CyberActionsHomeland,CyberActionsBagaria,US_dummy,Taiwan_dummy,SouthAfrica_dummy,filter_$,hacking_scale,gender_rec,binary_guess_pass,binary_access_account_view_files,binary_add_delete_change,binary_malware_use_delete,binary_private_harassment,binary_malware_use_access,binary_public_harassment,binary_write_malware,CyberTrespass,GAvai_FB,Total_Hour,Male,phy_foreign_target,cbr_foreign_target,phy_home_target,cbr_home_target,ENI_Factors,BGE_Factors,OGA_Factors,TECH_Factors,PIR_Factors,cbr_home_intensity,cbr_foreign_intensity,phy_home_intensity,phy_foreign_intensity,cyber_home_target,cyber_foreign_target
0,3,12,1,1,0,0,0,0,0.0,3,4,4,4,3,2,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0,1,0,1,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,0.0,0,2,0,2,0.41,1.37,0.07,1.69,1.32,medium,medium,medium,low,0,0
1,2,1,1,0,0,0,0,0,0.0,3,5,5,5,5,4,4,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0,1,0,1,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,7.0,0.0,0,2,0,2,-1.61,8.13,2.82,-2.62,-0.51,medium,medium,medium,medium,0,0
2,2,13,1,0,1,0,0,0,0.0,2,3,4,2,2,2,2,2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,3.0,1.0,1.0,0,1,0,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0,0,2,0,2,0.82,0.67,-1.53,2.15,0.30,medium,medium,medium,high,0,0
3,3,3,0,0,1,0,0,0,0.0,3,4,4,4,2,3,3,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,2.0,1.0,1.0,0,1,0,1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,0.0,0,2,0,2,0.47,-1.69,-0.10,-0.40,-1.91,medium,medium,medium,medium,0,0
4,2,1,1,0,0,0,0,0,0.0,2,5,4,1,2,3,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0,1,0,1,12.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,17.0,0.0,6.0,0.0,0,0,0,0,2.41,2.86,-1.96,2.56,-1.10,low,low,high,medium,0,0


In [36]:
X = dumy2[pred_vars_all]
y = dumy2['phy_foreign_target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
logmodel = LogisticRegression(max_iter=1000)
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logmodel.score(X_test, y_test)))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test,predictions))


Accuracy of logistic regression classifier on test set: 0.69
[[151   0]
 [ 68   1]]
              precision    recall  f1-score   support

           0       0.69      1.00      0.82       151
           1       1.00      0.01      0.03        69

    accuracy                           0.69       220
   macro avg       0.84      0.51      0.42       220
weighted avg       0.79      0.69      0.57       220

